In [1]:
cd("../.") 
pwd()

"/Users/farhadyar/Documents/Project_PTVAE/progs/github_repo/LatentSubgroups"

In [2]:

using Pkg
if isfile("Project.toml") && isfile("Manifest.toml")
    Pkg.activate(".")
end

Pkg.instantiate()

#FIXME make the including and using clean. you are including usings in irrelevant files. also including with revise you can check it including only one file and see it is running independently and then continue
using IJulia
using Revise

includet("../src/structs.jl")
includet("../GAN/GAN.jl")
includet("../src/transformations.jl")
includet("../visualization/visualization.jl")
includet("../src/load_data.jl")
includet("../src/report.jl")

  Activating environment at `~/Documents/Project_PTVAE/progs/github_repo/LatentSubgroups/Project.toml`


In [3]:
x, dataTypeArray, args = load_dataset("GAN")

args.cross_validation_flag = true
args.n_folds = 10

args.pre_transformation = false
args.AIQN = false 
args.scaling = true
args.scaling_method = "scaling"
args.user_description_on_run = "This run has been done on $(today()) at $(Dates.format(now(), "HH:MM"))... It is a GAN experiment"

if args.data_string == "toy"
    gen_input_dim = 2
    gen_hidden_dim = 10
    dis_hidden_dim = 10
    batch_size = 50
    epochs = 1000
    η = 1e-4
elseif args.data_string == "sim"
    gen_input_dim = 15
    gen_hidden_dim = 30
    dis_hidden_dim = 25
    batch_size = 50
    epochs = 2000
    η = 1e-5

else
    gen_input_dim = 15
    gen_hidden_dim = 50
    dis_hidden_dim = 40
    batch_size = 50
    epochs = 5000
    η = 1e-5
end



1.0e-5

In [4]:
Random.seed!(11)

if args.cross_validation_flag

    cross_val_run_path = string(args.current_run_path, "/$(args.n_folds)fold_cross_validation")
    mkdir(cross_val_run_path)

    cross_val_sets = create_cross_validation_sets(x, args.n_folds)

    for fold = 1:args.n_folds    

        train_set, val_set = cross_val_sets[fold]
            
        args.current_run_path = string(cross_val_run_path, "/fold_$(fold)")
    
        mkdir(args.current_run_path)
        preprocess_ps = preprocess_params(input_dim = args.input_dim)
        preprocessed_data, preprocess_ps = preprocess!(args, preprocess_ps, train_set, dataTypeArray)
        preprocessed_data_val, preprocess_ps = preprocess_test_data(args, preprocess_ps, val_set, dataTypeArray)



        # define GAN
        generator = Chain(Dense(gen_input_dim, gen_hidden_dim, leakyrelu), Dense(gen_hidden_dim, args.input_dim))
        discriminator = Chain(Dense(args.input_dim, dis_hidden_dim, leakyrelu), Dense(dis_hidden_dim, 1, σ))
        gan = GAN(generator, discriminator, randn, gen_input_dim, η, dataTypeArray, batch_size, epochs)

        # train gan
        gan, loss_array_gan, gen_error = train_GAN!(gan, preprocessed_data', train_set, preprocess_ps)

        save_gan_results(val_set, gan, preprocess_ps, args, [], [];  split_post_fix = "_val")

    end

end

Epoch 100: Discriminator loss = 13.162774502004645	 Generator loss = 85.82181058037014
Epoch 200: Discriminator loss = 12.434251255733688	 Generator loss = 81.13070779043963
Epoch 300: Discriminator loss = 10.391946359545534	 Generator loss = 90.80336678135971
Epoch 400: Discriminator loss = 10.722858786563357	 Generator loss = 95.41995005687225
Epoch 500: Discriminator loss = 10.895607189795953	 Generator loss = 100.73136631375704
Epoch 600: Discriminator loss = 11.370452114133222	 Generator loss = 101.41793042534252
Epoch 700: Discriminator loss = 11.959671840934877	 Generator loss = 101.30818510619285
Epoch 800: Discriminator loss = 12.72008978316898	 Generator loss = 101.7799599663633
Epoch 900: Discriminator loss = 13.95490582815875	 Generator loss = 99.01886778100865
Epoch 1000: Discriminator loss = 14.656265815336457	 Generator loss = 97.35966929572707
Epoch 1100: Discriminator loss = 14.736062014895737	 Generator loss = 96.803308262014
Epoch 1200: Discriminator loss = 15.336714